### Import Required Libraries and Set Up Environment Variables

In [4]:
# Dependencies
import requests
import time
from dotenv import load_dotenv
import os
import pandas as pd
import json
import os
from datetime import datetime
## Load the NASA_API_KEY from the env file
load_dotenv()
NASA_API_KEY = os.getenv('NASA_API_KEY')

# Prints API key to ensure it is loaded correctly
print(os.getenv("NASA_API_KEY")) 

z3afBLJBtzh2onHmXA0KJ88TWtfVkzfEnCARzbg7


### CME Data

In [8]:
# Set the base URL to NASA's DONKI API:
base_url = "https://api.nasa.gov/DONKI/"

# Set the specifier for CMEs:
CME = "CME"

# Search for CMEs published between a begin and end date
startDate = "2013-05-01"
endDate   = "2024-05-01"

# Build URL for CME
query_url_CME = f"https://api.nasa.gov/DONKI/CME?startDate={startDate}&endDate={endDate}&api_key={NASA_API_KEY}"


In [14]:
# Make a "GET" request for the CME URL and store it in a variable named cme_response
cme_response = requests.get(query_url_CME)

In [15]:
# Convert the response variable to json and store it as a variable named cme_json
cme_json = cme_response.json()

In [ ]:
# Preview ONLY the first element from the cme_json list you created in JSON format
# Do NOT print out the entire list
# Use json.dumps with argument indent=4 to format data
print(json.dumps(cme_json[:1], indent=4))

In [ ]:
# Convert cme_json to a Pandas DataFrame 

# Keep only the columns: activityID, startTime, linkedEvents


In [ ]:
# Notice that the linkedEvents column allows us to identify the corresponding GST
# Remove rows with missing 'linkedEvents' since we won't be able to assign these to GSTs


In [ ]:
# Notice that the linkedEvents sometimes contains multiple events per row
# Write a nested for loop that iterates first over each row in the cme DataFrame (using the index)
# and then iterates over the values in 'linkedEvents' 
# and adds the elements individually to a list of dictionaries where each row is one element 

# Initialize an empty list to store the expanded rows


# Iterate over each index in the DataFrame

    # Iterate over each dictionary in the list
    
        # Append a new dictionary to the expanded_rows list for each dictionary item and corresponding 'activityID' and 'startTime' value
      
# Create a new DataFrame from the expanded rows


In [ ]:
# Create a function called extract_activityID_from_dict that takes a dict as input such as in linkedEvents
# and verify below that it works as expected using one row from linkedEvents as an example
# Be sure to use a try and except block to handle errors

        # Log the error or print it for debugging




In [ ]:
# Apply this function to each row in the 'linkedEvents' column (you can use apply() and a lambda function)
# and create a new column called 'GST_ActivityID' using loc indexer:


In [11]:
# Remove rows with missing GST_ActivityID, since we can't assign them to GSTs:


In [ ]:
# print out the datatype of each column in this DataFrame:


In [ ]:
# Convert the 'GST_ActivityID' column to string format 

# Convert startTime to datetime format  

# Rename startTime to startTime_CME and activityID to cmeID

# Drop linkedEvents

# Verify that all steps were executed correctly


In [ ]:
# We are only interested in CMEs related to GSTs so keep only rows where the GST_ActivityID column contains 'GST'
# use the method 'contains()' from the str library.  


### GST Data

In [15]:
# Set the base URL to NASA's DONKI API:
base_url = "https://api.nasa.gov/DONKI/"

# Set the specifier for Geomagnetic Storms (GST):
GST = "GST"

# Search for GSTs between a begin and end date
startDate = "2013-05-01"
endDate   = "2024-05-01"

# Build URL for GST


In [16]:
# Make a "GET" request for the GST URL and store it in a variable named gst_response


In [ ]:
# Convert the response variable to json and store it as a variable named gst_json


In [ ]:
# Preview ONLY the first element from the gst_json list you created in JSON format
# Do NOT print out the entire list
# Use json.dumps with argument indent=4 to format data


In [ ]:
# Convert gst_json to a Pandas DataFrame  

# Keep only the columns: gstID, startTime, linkedEvents


In [ ]:
# Notice that the linkedEvents column allows us to identify the corresponding CME
# Remove rows with missing 'linkedEvents' since we won't be able to assign these to CME


In [20]:
# Notice that the linkedEvents sometimes contains multiple events per row
# Use the explode method to ensure that each row is one element. Ensure to reset the index and drop missing values.


In [ ]:
# Apply the extract_activityID_from_dict function to each row in the 'linkedEvents' column (you can use apply() and a lambda function)
# and create a new column called 'CME_ActivityID' using loc indexer:

# Remove rows with missing CME_ActivityID, since we can't assign them to CMEs:


In [ ]:
# Convert the 'CME_ActivityID' column to string format 

# Convert the 'gstID' column to string format 

# Convert startTime to datetime format  

# Rename startTime to startTime_GST 

# Drop linkedEvents

# Verify that all steps were executed correctly


In [ ]:
# We are only interested in GSTs related to CMEs so keep only rows where the CME_ActivityID column contains 'CME'
# use the method 'contains()' from the str library.  


### Merge both datatsets

In [ ]:
# Now merge both datasets using 'gstID' and 'CME_ActivityID' for gst and 'GST_ActivityID' and 'cmeID' for cme. Use the 'left_on' and 'right_on' specifiers.


In [ ]:
# Verify that the new DataFrame has the same number of rows as cme and gst


### Computing the time it takes for a CME to cause a GST

In [ ]:
# Compute the time diff between startTime_GST and startTime_CME by creating a new column called `timeDiff`.


In [ ]:
# Use describe() to compute the mean and median time 
# that it takes for a CME to cause a GST. 


### Exporting data in csv format

In [28]:
# Export data to CSV without the index
